In [89]:
import json
import os
from osmxtract import overpass, location
import geopandas as gpd
import requests
import json
shapefile_usosuolo="http://webapps.comune.trento.it/cartografia/gis/dbexport?db=base&sc=uso_suolo&ly=usosuolo_view&fr=shp"
segnalazioni_sensorcivico="https://sensorcivico.comune.trento.it/sensor/data?contentType=geojson"
confini_trento=".."+ os.sep + "data" + os.sep + "confini_trento.shp"
url_dettagli_sensorcivico="https://sensorcivico.comune.trento.it/sensor/data?contentType=marker&id=ID" #sostituire ID con l'ID della segnalazione

In [90]:
confini = gpd.read_file(confini_trento,driver="ESRI Shapefile")

In [91]:
#aree_verdi_urbane = usosuolo[usosuolo.fumetto=='Aree verdi urbane'].plot()

In [92]:
bounds = (confini.bounds.miny[0],confini.bounds.minx[0], confini.bounds.maxy[0],confini.bounds.maxx[0])

In [93]:
query = overpass.ql_query(bounds,tag='leisure', values=['park'])

In [94]:
response = overpass.request(query)

In [95]:
leisure_park_geojson = overpass.as_geojson(response, 'polygon')

In [96]:
leisure_park_osm = gpd.GeoDataFrame.from_features(leisure_park_geojson)

In [97]:
r = requests.get(segnalazioni_sensorcivico)

In [98]:
data = r.json()

In [99]:
data.keys()

dict_keys(['type', 'features'])

In [100]:
for idx in range(len(data['features'])):
    data['features'][idx]['properties']['id'] = data['features'][idx]['id']

In [101]:
for k in range(len(data['features'])):
    data['features'][k]['geometry']['coordinates'][0] = float(data['features'][k]['geometry']['coordinates'][0])
    data['features'][k]['geometry']['coordinates'][1] = float(data['features'][k]['geometry']['coordinates'][1])


In [102]:
poi_sensorcivico = gpd.GeoDataFrame.from_features(data)

In [103]:
park_index = []
poi_index = []
for idxpoi, poi in poi_sensorcivico.iterrows():
    for idxpark, park in leisure_park_osm.iterrows():
        if (poi.geometry.within(park.geometry)):
            park_index.append(idxpark)
            poi_index.append(idxpoi)
            break

In [104]:
def getDesc(id):
    r = requests.get(url_dettagli_sensorcivico.replace("ID",str(id)))
    response = r.json()
    text = "" 
    if len(response) > 0:
        text = r.json()
    return text

In [105]:
poi_sensorcivico_parchi = poi_sensorcivico[poi_sensorcivico.index.isin(poi_index)]

In [106]:
poi_sensorcivico_parchi

,geometry,id,name,popupContent,type
14,POINT (11.128603 46.068327),17236,Piastra di piazza Venezia lasciata a se stessa...,<em>Loading...</em>,reclamo
15,POINT (11.06643 46.087683),17233,Segnalazione su Shelly App,<em>Loading...</em>,segnalazione
61,POINT (11.12855 46.068281),16995,Allestimento e rifacimento del campetto da Bas...,<em>Loading...</em>,segnalazione
102,POINT (11.11605 46.103043),16836,Sostegni altalena bambini rotti,<em>Loading...</em>,segnalazione
128,POINT (11.11245 46.10744),16682,Albero,<em>Loading...</em>,segnalazione
140,POINT (11.126747 46.063666),16644,Parco Santa Chiara,<em>Loading...</em>,segnalazione
175,POINT (11.137615 46.054784),16496,danni area cani Gocciadoro,<em>Loading...</em>,segnalazione
215,POINT (11.114763 46.097988),16230,Esercizi pericolosi,<em>Loading...</em>,segnalazione
227,POINT (11.12855 46.068244),16176,Campetto basket,<em>Loading...</em>,suggerimento
242,POINT (11.113358 46.059578),16082,Fontanelle senza acqua (Le Albere),<em>Loading...</em>,segnalazione


In [107]:
poi_sensorcivico_parchi['desc'] = poi_sensorcivico_parchi['id'].apply(lambda x: getDesc(x))

/home/napo/dev/scraping_comune_trento/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [108]:
poi_sensorcivico_parchi.to_file('poi_su_parchi.geojson', driver="GeoJSON")